In [1]:
import TSD
import altair as alt
import pandas as pd
import numpy as np
import math
from datetime import datetime
from dateutil.relativedelta import relativedelta
import random
from altair.expr import datum, if_

In [2]:
TSD.df_merged

,dob,gender,city,colonia,state,timein,timeout,clinic_id,patient_id,location,start,end,Category
0,2005-08-09,f,punta colonet,Unlisted,BC,2021-05-07 09:44:49.715394,2021-05-07 09:44:49.715421,15,1,Thousand Smiles Ensenada,2021-05-01,2021-05-11,[Unlisted]
1,2005-08-09,f,punta colonet,Unlisted,BC,2021-08-06 09:47:35.775716,2021-08-06 09:47:35.775741,16,1,Thousand Smiles Ensenada,2021-08-01,2021-08-08,[Unlisted]
2,2005-08-09,f,punta colonet,Unlisted,BC,2022-05-06 10:01:49.350689,2022-05-06 10:01:49.350714,23,1,Ensenada,2022-05-06,2022-05-08,[Unlisted]
3,2005-08-09,f,punta colonet,Unlisted,BC,2022-11-04 10:19:36.302828,2022-11-04 10:19:36.302844,25,1,Ensenada,2022-11-04,2022-11-06,[Unlisted]
4,2005-08-09,f,punta colonet,Unlisted,BC,2023-02-03 08:59:22.584125,2023-02-03 08:59:22.584141,30,1,Ensenada,2023-02-03,2023-02-05,[Unlisted]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,2012-06-04,f,Ensenada,Villas del Sol,BC,2023-08-05 10:24:36.296644,2023-08-05 10:24:36.296659,32,834,Ensenada,2023-08-04,2023-08-06,[Born With Cleft Palate]
1022,2011-04-07,f,Ensenada,Revolucion,BC,2023-08-05 10:41:49.432398,2023-08-05 10:41:49.432414,32,835,Ensenada,2023-08-04,2023-08-06,[Diabetes]
1023,2013-05-19,m,ensenada,Valle Dorado,BC,2023-08-05 11:11:44.717322,2023-08-05 11:11:44.717336,32,836,Ensenada,2023-08-04,2023-08-06,[Unlisted]
1024,2012-06-28,f,Ensenada,Calls Dorado,BC,2023-08-05 11:12:15.452107,2023-08-05 11:12:15.452120,32,837,Ensenada,2023-08-04,2023-08-06,[Trouble Speaking]


In [3]:
def compute_age(row):
    d1 = row["start"]
    d2 = row["dob"]
    diff_in_years = relativedelta(d1, d2)
    return diff_in_years.years

merged_df = TSD.df_merged
merged_df["age_in_years"] = merged_df.apply(lambda row: compute_age(row), axis=1)
age_row = merged_df["age_in_years"]

In [4]:
#calculate average per clinic historically
clinic_counts = merged_df.groupby(['clinic_id','age_in_years']).size().reset_index(name='count')
avg_clinic = clinic_counts.groupby('age_in_years').mean().drop('clinic_id', axis=1).reset_index()
avg_clinic.age_in_years = avg_clinic.age_in_years // 5
grouped_avg = avg_clinic.groupby('age_in_years').sum().reset_index()
grouped_avg.age_in_years = grouped_avg.age_in_years * 5

#find number of clinics
total_clinics = merged_df.clinic_id.nunique()

#create a list of dataframes, one for each clinic


In [5]:
#this randomizes the category data so that it no longer matches by patient, 
#but gives the correct perecentage of each condition
    

clinics = {}
totals_by_age_clinic = {}


for index, row in merged_df.iterrows():
    # Create a tuple of (clinic_id, age_in_years) to identify clinics uniquely
    if row.age_in_years == 0:
        clinic_key = (row.clinic_id, 5, row.gender)
    else:
        clinic_key = (row.clinic_id, math.ceil(row.age_in_years / 5) * 5, row.gender)
        
    for cat in row.Category:
        new_key = (clinic_key[0],clinic_key[1],clinic_key[2],cat)
        if new_key in clinics:
            clinics[new_key] += 1
        else:
            clinics[new_key] = 1
    
    #calculate totals_by_age_clinic
    second_key = (clinic_key[0],clinic_key[1])
    if second_key in totals_by_age_clinic:
        totals_by_age_clinic[second_key] += 1
    else:
        totals_by_age_clinic[second_key] = 1                     


dummy_df = pd.DataFrame()
rows = []

##percent of total for age/clinic


for key, count in clinics.items():
    new_row = {}
    new_row['clinic_id'] = key[0]
    new_row['age'] = key[1]
    new_row['Category'] = key[3]
    new_row['Count'] = count
    rows.append(new_row)
    percent = 100*count/totals_by_age_clinic[(key[0],key[1])]
    #check for other gender version and add together
    gender = key[2]
    key_opp = (key[0],key[1],'m' if gender == 'f' else 'f',key[3])
    key_opp_count = 0
    if key_opp in clinics:
        key_opp_count = clinics[key_opp]
        percent = percent + 100*key_opp_count/totals_by_age_clinic[(key_opp[0],key_opp[1])]
    new_row['Percent'] = str(round(percent,2))+"%"
    if gender == 'f':
        gender_text = str(key_opp_count)+"/"+str(count)
    else:
        gender_text = str(count)+"/"+str(key_opp_count)
    new_row['gender'] = gender_text
dummy_df = pd.DataFrame(rows)
#clinic_id, age, category, percent of total for age/clinic, Gender M/F

#duplicate rows not sure how else do do this
rows = []
for row in dummy_df.itertuples():
    for _ in range(row.Count):
        rows.append(row)
final_df = pd.DataFrame(rows)
final_df = final_df.rename(columns={'gender': 'Gender M/F'})
final_df['age'] = final_df['age'] - 1

final_df


,Index,clinic_id,age,Category,Count,Percent,Gender M/F
0,0,15,14,Unlisted,1,100.0%,0/1
1,1,16,14,Unlisted,12,85.0%,5/12
2,1,16,14,Unlisted,12,85.0%,5/12
3,1,16,14,Unlisted,12,85.0%,5/12
4,1,16,14,Unlisted,12,85.0%,5/12
...,...,...,...,...,...,...,...
1317,508,32,29,Bleeding Problems,1,50.0%,1/1
1318,509,32,29,Trouble Speaking,1,25.0%,1/0
1319,510,32,29,Trouble Eating,1,25.0%,1/0
1320,511,32,14,Epilepsy,1,3.85%,0/1


In [6]:
#setup dates in final_df for Dropdown
date_format = "%d %B %Y"

final_df['clinic_dates'] = final_df['clinic_id']
df = TSD.df_clinic.copy()
replace_dict = df.set_index('id')['start'].to_dict()
final_df['clinic_dates'] = final_df['clinic_dates'].replace(replace_dict)
final_df['clinic_dates'] = pd.to_datetime(final_df['clinic_dates']).dt.strftime(date_format)

clinic_dates = final_df.clinic_dates.unique()

# Convert date strings to datetime objects
date_objects = np.array([datetime.strptime(date, date_format) for date in clinic_dates])

# Sort datetime objects in descending order
sorted_dates = np.sort(date_objects)[::-1]

# Convert sorted datetime objects back to date strings
clinic_dates = [date.strftime(date_format) for date in sorted_dates]



# A dropdown filter
clinic_dropdown = alt.binding_select(options=clinic_dates, name="Clinic Date:")
clinic_select = alt.selection_point(fields=['clinic_dates'], bind=clinic_dropdown, \
                                    value=clinic_dates[0])

#Chart most recent clinic with line showing historical average
line = alt.Chart(grouped_avg).mark_line(color='blue', strokeOpacity=0.35).encode(
    alt.X('age_in_years:Q',title='Age in Years',bin = alt.Bin(maxbins=20)),
    y = 'count:Q',
)



base = alt.Chart(final_df).encode (
       x =    alt.X('age', bin = alt.Bin(maxbins=20))   
)
bars = base.mark_bar().encode(
    y = alt.Y('count()',title='Number of Patients'),
    color=alt.Color('Category:N', title='Category'),
    tooltip=['Category:N','Percent:N','Gender M/F:N']).add_params(
    clinic_select
).transform_filter(
    clinic_select
)

chart = alt.layer(bars, line).properties(
    width=700,
    height=500)

chart.save('TS-Demographics.html')
chart


alt.LayerChart(...)